## CoReComm  
`Collaborative Recommendation with Community Detection (CoReComm)`

⚠️ This Jupyter notebook aims to conduct an experimental study to evaluate the performance of various collaborative filtering algorithms when combined with community detection techniques.
The algorithms will be evaluated based on RMSE, MSE and MAE metrics and the results will be saved in `.csv` file for further analysis.

### ⚙️ Environment settings

In [18]:
"""
    Importing needed libs
"""

'\n    Importing needed libs\n'

In [19]:
import os
import time
import numpy as np
import pandas as pd
from typing import List
from surprise import (
    accuracy,
    Reader,
    Dataset,
    CoClustering,
    KNNBasic,
    NMF,
    SVD,
    SlopeOne,
    NormalPredictor
)
from surprise.model_selection.split import ShuffleSplit
from surprise.trainset import Trainset
import networkx as nx
import cdlib.algorithms as cd_alg
import karateclub as kc
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [20]:
"""
    Setting up functions
"""

'\n    Setting up functions\n'

In [21]:
def uncouple(train_set: Trainset, test_set: list):
    """
    Description:
        It takes in a Trainset-Surprise object and 
        a list of test set data, and returns two pandas 
        dataframes: one containing the training set 
        data, and the other containing the test set 
        data.
    Input: 
        train_set: a Trainset object containing the 
        training set data. 
        test_set: a list containing the test set data
    Output:
        df_train: a pandas dataframe containing the 
        training set data, with columns 'uid', 'iid' 
        and 'rating'
        df_test: a pandas dataframe containing the test 
        set data, with columns 'uid', 'iid', and 'rating'
    """
    iterator = train_set.all_ratings()
    df_train = pd.DataFrame(train_set.all_ratings(), columns=['uid', 'iid', 'rating'])
    df_test = pd.DataFrame.from_records(test_set, columns = ['uid', 'iid', 'rating'])
    
    return df_train, df_test


def communitarian_recommendation_graph(
        n_user=100,
        n_comm=10,
        n_item=100,
        n_cat=10,
        like_prob=.4,
        like_strength=.5,
        debug=False,
    ):

    user_comm_gt = np.random.choice(n_comm, n_user)
    user_comm_gt.sort()
    bincount_user = np.bincount(user_comm_gt, minlength=n_comm)

    if debug:
        display('user', user_comm_gt, bincount_user)

    item_cat_gt = np.random.choice(n_cat, n_item)
    item_cat_gt.sort()
    bincount_item = np.bincount(item_cat_gt, minlength=n_cat)

    if debug:
        display('item', item_cat_gt, bincount_item)

    L = np.random.choice(2, (n_comm, n_cat), p=[1-like_prob, like_prob])

    L_prime = like_strength * L + (1-like_strength)/2

    P = np.block([
        [ np.zeros((n_comm,n_comm)) , L_prime                 ],
        [ L_prime.T                 , np.zeros((n_cat,n_cat)) ]
    ])

    if debug:
        display('like', L, L_prime, P)

    sbg = nx.stochastic_block_model(
        np.hstack((bincount_user, bincount_item)),
        P
    )

    if debug:
        try:
            nx.draw_networkx(
                sbg,
                nx.bipartite_layout(sbg, nx.bipartite.sets(sbg)[0]),
                node_color=np.hstack((user_comm_gt, item_cat_gt))
            )
        except nx.AmbiguousSolution as e:
            print(e)            

        plt.show()

    for user, item, data in sbg.edges(data=True):
        data['weight'] = np.random.randint(1,4) + 2*L[user_comm_gt[user], item_cat_gt[item-n_user]]

    if debug:
        display(np.mean([w['weight'] for user, item, w in sbg.edges(data=True) if L[user_comm_gt[min(user, item)], item_cat_gt[max(user, item)-n_user]]]))
        display(np.mean([w['weight'] for user, item, w in sbg.edges(data=True) if not L[user_comm_gt[min(user, item)], item_cat_gt[max(user, item)-n_user]]]))
        
    return sbg, user_comm_gt, item_cat_gt


def wf(G, u, v):
    w = 0    
    for nbr in set(G[u]) & set(G[v]):
        w += (G[u][nbr]['weight'] - G[v][nbr]['weight']) ** 2
    return 1/(1 + w)


def internal_density(graph , community):
    """
    Calculates the internal density of a community in a graph.
    :param graph: The input graph.
    :type graph: NetworkX graph object.
    :param community: The community of interest.
    :type community: List of nodes.

    :return: The internal density of the community.
    :rtype: float between 0 and 1.
    """
    num_edges = 0
    num_nodes = len(community)
    for u, v in graph.edges(community):
        num_edges += 1
    max_edges = num_nodes * (num_nodes - 1) / 2
    if max_edges > 0:
        return num_edges / max_edges
    else:
        return 0.0


In [22]:
"""
    Setting up experiment algorithms 
"""

'\n    Setting up experiment algorithms \n'

In [23]:
algos_recommendation = {
    'NormalPredictor': NormalPredictor(),
    'SVD': SVD(verbose=False),
    'k-NN': KNNBasic(verbose=False), 
    'NMF': NMF(verbose=False), 
    'Co-Clustering': CoClustering(verbose=False),
    'SlopeOne': SlopeOne()
}

communities_detectors = {
    'Not-Applicable': None,
    'Louvain': nx.community.louvain_communities,
    'Paris': cd_alg.paris
}

### 🖥️ Getting data

In [26]:
dataset = Dataset.load_builtin('ml-100k')

In [27]:
pd.DataFrame({'# Users': [len(set([user_id for (user_id, _, _, _) in dataset.raw_ratings]))],
              '# Itens': [len(set([item_id for (_, item_id, _, _) in dataset.raw_ratings]))],
              '# Ratings': [len(dataset.raw_ratings)]
              }
             )

,# Users,# Itens,# Ratings
0,943,1682,100000


### 🏃 Running experiment

In [28]:
reader = Reader(rating_scale=(1, 5))

ml100k_df = pd.DataFrame(dataset.raw_ratings)[[0,1,2]].rename(columns={2:'weight'})
ml100k_df = ml100k_df.astype(int)
ml100k_df[[0,1]] -= ml100k_df[[0,1]].min()
n_users = ml100k_df[0].max() + 1
ml100k_df[1] += ml100k_df[0].max() + 1
ml100k_df.rename(columns={0:'uid', 1:'iid', 'weight': 'rating'}, inplace=True)
ml100k_df
dataset = Dataset.load_from_df(ml100k_df[['uid', 'iid', 'rating']], reader)


In [29]:
pd.DataFrame({'# Users': [len(set([user_id for (user_id, _, _, _) in dataset.raw_ratings]))],
              '# Itens': [len(set([item_id for (_, item_id, _, _) in dataset.raw_ratings]))],
              '# Ratings': [len(dataset.raw_ratings)]
              }
             )

,# Users,# Itens,# Ratings
0,943,1682,100000


In [30]:
results=[]
tracking=[]
reader = Reader(rating_scale=(1, 5))

for test_size in tqdm([0.25, 0.1, 0.01], desc='📊 Test Size Progress', leave=True):
        shuffle_split = ShuffleSplit(n_splits=5, test_size=test_size)
        split_id = 1
        for trainset, testset in tqdm(shuffle_split.split(dataset), desc='➗ Splits Progress', leave=False):
            for detector_name, community_detector in tqdm(communities_detectors.items(), desc='🕸️ Community Detector Progress', leave=False):
                if community_detector != None:
                    trainpd, testpd = uncouple(trainset, testset)
                    users_id = sorted(
                            list(
                                set(
                                    trainpd['uid']
                                    )
                                )
                            )
                    temp = trainpd.copy()
                    temp.rename(columns={
                        'uid': 'source',
                        'iid': 'target',
                        'rating': 'weight'
                    }, inplace=True)
                    crg = nx.from_pandas_edgelist(temp, source='source', target='target', edge_attr='weight')
                    g = nx.bipartite.generic_weighted_projected_graph(crg, 
                                                users_id, wf
                                                )
                    if detector_name == 'Louvain':
                        coms = community_detector(g.copy())
                    else:
                        coms = community_detector(g.copy()).communities

                    densities = {}
                    for i, com in enumerate(coms):
                        density = internal_density(g, com)
                        densities[i] = density
                    densities = dict(sorted(densities.items(), 
                                            key=lambda x: x[1], reverse=True)
                                        )
                for algo_name, algo in  tqdm(algos_recommendation.items(), desc='👾 Algorithm Progress', leave=False):
                    if community_detector == None:
                        start_time = time.time()
                        algo.fit(trainset)
                        training_time = time.time() - start_time
                        start_time = time.time()
                        predictions = algo.test(testset)
                        testing_time = time.time() - start_time
                        rmse_value = accuracy.rmse(predictions, verbose=False)
                        mse_value = accuracy.mse(predictions, verbose=False)
                        mae_value = accuracy.mae(predictions, verbose=False)
                        result_dict = {
                            'community_detector': detector_name,
                            'used_communities': np.nan,
                            'algorithm_rec': algo_name,
                            'test_size': test_size,
                            'split_id': split_id,
                            'rmse': rmse_value,
                            'mse': mse_value,
                            'mae': mae_value
                            }
                        tracking_dict = {
                            'community_detector': detector_name,
                            'used_communities': np.nan,
                            'community': np.nan,
                            'algorithm_rec': algo_name,
                            'test_size': test_size,
                            'split_id': split_id,
                            'train_runtime': training_time,
                            'test_runtime': testing_time,    
                        }
                        results.append(result_dict)
                        tracking.append(tracking_dict)
                    else:
                        all_predictions = []
                        for n in range(1, len(densities) + 1):
                            most_dense_communities = list(
                                densities.keys()
                                )[:n]
                            for m in most_dense_communities:
                                community=coms[m]
                                train_community = trainpd[trainpd['uid'].isin(community)]
                                test_community = testpd[testpd['uid'].isin(community)]
                                train_surprise = Dataset.load_from_df(train_community[['uid', 'iid', 'rating']], reader)
                                train_surprise = train_surprise.build_full_trainset()
                                test_community = test_community.astype(
                                    {'uid':'float', 'iid':'float', 'rating':'float'}
                                    )
                                test_surprise = list(test_community.itertuples(index=False, name=None))
                                
                                start_time = time.time()
                                algo.fit(train_surprise)
                                training_time = time.time() - start_time
                                start_time = time.time()
                                predictions = algo.test(test_surprise) 
                                testing_time = time.time() - start_time
                                all_predictions.extend(predictions)
                                tracking_dict = {
                                    'community_detector': detector_name,
                                    'used_communities': n,
                                    'community': m,
                                    'algorithm_rec': algo_name,
                                    'test_size': test_size,
                                    'split_id': split_id,
                                    'train_runtime': training_time,
                                    'test_runtime': testing_time,    
                                }
                                tracking.append(tracking_dict)
                                
                            if len(most_dense_communities) < len(densities):
                                train_community = trainpd[~trainpd['uid'].isin(community)]
                                test_community = testpd[~testpd['uid'].isin(community)]
                                train_surprise = Dataset.load_from_df(train_community[['uid', 'iid', 'rating']], reader)
                                train_surprise = train_surprise.build_full_trainset()
                                test_community = test_community.astype(
                                    {'uid':'float', 'iid':'float', 'rating':'float'}
                                    )
                                test_surprise = list(test_community.itertuples(index=False, name=None))
                                start_time = time.time()
                                algo.fit(train_surprise)
                                training_time = time.time() - start_time
                                start_time = time.time()
                                predictions = algo.test(test_surprise) 
                                testing_time = time.time() - start_time
                                all_predictions.extend(predictions)
                                tracking_dict = {
                                    'community_detector': detector_name,
                                    'used_communities': n,
                                    'community': 99, # 99 means `all the others communities`
                                    'algorithm_rec': algo_name,
                                    'test_size': test_size,
                                    'split_id': split_id,
                                    'train_runtime': training_time,
                                    'test_runtime': testing_time,    
                                }
                                tracking.append(tracking_dict)
                                    
                            rmse_value = accuracy.rmse(all_predictions, verbose=False)
                            mse_value = accuracy.mse(all_predictions, verbose=False)
                            mae_value = accuracy.mae(all_predictions, verbose=False)
                            result_dict = {
                                    'community_detector': detector_name,
                                    'used_communities': n,  
                                    'algorithm_rec': algo_name,
                                    'test_size': test_size,
                                    'split_id': split_id,
                                    'rmse': rmse_value,
                                    'mse': mse_value,
                                    'mae': mae_value
                                }
                            results.append(result_dict)            
            split_id += 1
            
            
notebook_dir = os.getcwd()
outputs_dir = notebook_dir.replace('notebooks', 'outputs')

df_results = pd.DataFrame(results)
file_name = f'real_results.csv'
file_path = os.path.join(outputs_dir, file_name)
df_results.to_csv(file_path, index=False)

df_tracking = pd.DataFrame(tracking)
file_name = f'real_tracking.csv'
file_path = os.path.join(outputs_dir, file_name)
df_tracking.to_csv(file_path, index=False)          

📊 Test Size Progress:   0%|          | 0/3 [00:00<?, ?it/s]

➗ Splits Progress: 0it [00:00, ?it/s]

🕸️ Community Detector Progress:   0%|          | 0/3 [00:00<?, ?it/s]

👾 Algorithm Progress:   0%|          | 0/6 [00:00<?, ?it/s]

👾 Algorithm Progress:   0%|          | 0/6 [00:00<?, ?it/s]

...

👾 Algorithm Progress:   0%|          | 0/6 [00:00<?, ?it/s]

🕸️ Community Detector Progress:   0%|          | 0/3 [00:00<?, ?it/s]

👾 Algorithm Progress:   0%|          | 0/6 [00:00<?, ?it/s]

👾 Algorithm Progress:   0%|          | 0/6 [00:00<?, ?it/s]

👾 Algorithm Progress:   0%|          | 0/6 [00:00<?, ?it/s]